# Set up

In [1]:
#import necessary packages

import os
import glob
from nilearn import surface
import numpy as np
import nibabel as nib
import usefulFunctions as uf

In [2]:
# flag to save files
save_files = False

#set paths
base_dir = f'{os.path.dirname(os.getcwd())}/'
roi_dir = f'{base_dir}data/labels/average/allData/'
fs_mesh = f'{base_dir}data/surface_meshes/fsaverage/lh.inflated'
out_dir = f'{base_dir}analysis/overlap_maps/'
fig_dir = f'{base_dir}figures/overlap_ss/'

# define subject lists
subs = uf.subject_list(roi_dir)
id_cutoff = 1000 # split between child and adult ID #s
child_subs = [sub for sub in subs if int(sub.split('-')[1]) < id_cutoff]
adult_subs = [sub for sub in subs if int(sub.split('-')[1]) >= id_cutoff]
sub_groups = [child_subs,adult_subs]

# set variables
sub_group_names = ['child','adult']
ROIs = ['VWFA', 'FFA']

# Create heat and probability maps of overlapping ROIs

In [4]:
# load the surface mesh
fsavg_surf = surface.load_surf_mesh(fs_mesh)

#loop through group and lables to grab all ROIs
for group_indx, group in enumerate(sub_groups):
    for roi in ROIs:

        # create mesh with 0 for every index
        roi_overlap = np.zeros(len(fsavg_surf[0]))

        # keep track of the number of participants with an roi
        roi_count = 0
        
        for sub in group:

            # append labels to list if they exists
            label_paths = glob.glob(f'{roi_dir}{sub}/{sub}_*-v-ALL_{roi}*_LH.label')
            for label_path in label_paths:
                if os.path.exists(label_path):
                    label = surface.load_surf_data(label_path)

                    # increment roi_overlap for each index in each label
                    for idx in label:
                        roi_overlap[idx] += 1
            
            # increment roi_count for each subject with an roi
            if len(label_paths) > 0:
                roi_count += 1

        # calcualte roi probability map
        prob_map = roi_overlap/roi_count
        
        # print for assigning visualization later
        print(np.max(prob_map))
        print(f'{sub_group_names[group_indx]} {roi} {roi_count}')
    
        # # save maps
        if save_files:
            nib.freesurfer.io.write_morph_data(f'{out_dir}{sub_group_names[group_indx]}_{roi}_overlap.curv', roi_overlap)
            nib.freesurfer.io.write_morph_data(f'{out_dir}{sub_group_names[group_indx]}_{roi}_probability.curv', prob_map)

0.2571428571428571
child VWFA 70
0.6707317073170732
child FFA 82
0.46153846153846156
adult VWFA 13
0.7857142857142857
adult FFA 14


## Get freeview screenshot

In [ ]:
# specify camera aesthetics
viewsize = ' -viewsize 600 665'
camera = '--camera Azimuth 0 Elevation -70 Roll 270'

# set color wheel thresholds
min_thresh = 0.0001
max_thresh = 0.8 # based on max value from all probability maps

# set fig directory
fig_dir = '/Users/jamiel12/scratch/jamie/fROI_methods/figures/overlap_ss/'

# loop through group 
for group in sub_group_names:
    for roi in ROIs:
        # grab the map path
        map = f'{out_dir}{group}_{roi}_probability.curv'
    
        # build the command
        cmd = f'freeview -f {fs_mesh}:curvature_method=binary:overlay={map}:overlay_color=colorwheel:overlay_threshold={min_thresh},{max_thresh} {camera} -ss {fig_dir}{group}_{roi}_probability.png {viewsize}'
    
        # run command
        print(cmd)
        # os.system(cmd)    

# Create indicator map to define overlap between VWFA & FFA labels

In [5]:
# load the surface mesh
fsavg_surf = surface.load_surf_mesh(fs_mesh)

#loop through group to grab overlap maps
for group_indx, group in enumerate(sub_groups):

    # load VWFA & FFA overlap maps
    VWFA_overlap = surface.load_surf_data(f'{out_dir}{sub_group_names[group_indx]}_VWFA_overlap.curv')
    FFA_overlap = surface.load_surf_data(f'{out_dir}{sub_group_names[group_indx]}_FFA_overlap.curv')

    # assign an vertex with a VWFA as a 1
    VWFA_indices = []
    for index in VWFA_overlap:
        if index == 0:
            VWFA_indices.append(0.0)
        elif index >0:
            VWFA_indices.append(1.0)

    # assign an vertex with a FFA as a 2
    FFA_indices = []
    for index in FFA_overlap:
        if index == 0:
            FFA_indices.append(0.0)
        elif index >0:
            FFA_indices.append(2.0)

    # add maps so anywhere VWFA & FFA overlap == 3
    VWFA_FFA_overlap = np.array(VWFA_indices) + np.array(FFA_indices)

    # save map
    if save_files:
        nib.freesurfer.io.write_morph_data(f'{out_dir}{sub_group_names[group_indx]}_VWFA-FFA_overlap.curv', VWFA_FFA_overlap)

## Get freeview screenshot

In [ ]:
# specify camera aesthetics
viewsize = ' -viewsize 600 665'
camera = '--camera Azimuth 0 Elevation -70 Roll 270'

custom_colors = '1,16,255,22,2,249,12,233,3,255,255,0' # 'val,r,g,b,val2,r2,g2,b2..''

# set fig directory
fig_dir = '/Users/jamiel12/scratch/jamie/fROI_methods/figures/overlap_ss/'

# loop through group 
for group in sub_group_names:
    for roi in ROIs:
        # grab the map path
        map = f'{out_dir}{group}_VWFA-FFA_overlap.curv'
    
        # build the command
        cmd = f'freeview -f {fs_mesh}:curvature_method=binary:overlay={map}:overlay_custom={custom_colors} {camera} -ss {fig_dir}{group}_VWFA-FFA_overlap.png {viewsize}'
    
        # run command
        print(cmd)
        # os.system(cmd)    

# Calculate overlap vertices

In [6]:
# define groups
groups = ['adult','child']

# 1 = VWFA, 2 = FFA, 3 = both

# loop through groups to calculate vertex counts
for group in groups:
    print(group)
    # load the overlap map
    overlap_map = surface.load_surf_data(f'{out_dir}{group}_VWFA-FFA_overlap.curv')

    # count the number of vertices for each value
    print(f'VWFA {len(np.where(overlap_map == 1.0)[0])} {len(np.where(overlap_map == 1.0)[0])/len(overlap_map)}%')
    print(f'FFA {len(np.where(overlap_map == 2.0)[0])} {len(np.where(overlap_map == 2.0)[0])/len(overlap_map)}%')
    print(f'Both {len(np.where(overlap_map == 3.0)[0])} {len(np.where(overlap_map == 3.0)[0])/len(overlap_map)}%')
    print(f'None {len(np.where(overlap_map == 0.0)[0])} {len(np.where(overlap_map == 0.0)[0])/len(overlap_map)}%')
    


adult
VWFA 649 0.003961133286947181%
FFA 1369 0.008355610893421711%
Both 2026 0.01236557170932972%
None 159798 0.9753176841103014%
child
VWFA 962 0.005871510357539581%
FFA 2450 0.01495343074425361%
Both 3093 0.01887794338448017%
None 157337 0.9602971155137267%
